In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from nltk.stem import PorterStemmer

In [3]:
df=pd.read_csv('spam_ham_dataset.csv')
df

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


In [6]:
text=df['text'][0]
print(text)

Subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes .


In [7]:
#Lower case whole text
text=text.lower()
print(text)

subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes .


In [8]:
#removing digits (\d+)
text=re.sub(r'\d+','',text)
print(text)

subject: enron methanol ; meter # : 
this is a follow up to the note i gave you on monday ,  /  /  { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes .


In [10]:
#removing punctuation
text=''.join([i for i in text if i not in string.punctuation])
print(text)

subject enron methanol  meter   
this is a follow up to the note i gave you on monday        preliminary
flow data provided by daren  
please override pop  s daily volume  presently zero  to reflect daily
activity you can obtain from gas control 
this change is needed asap for economics purposes 


In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
#remove extra space from the begining or ending of sent
tetx=text.strip()
print(text)

subject enron methanol  meter   
this is a follow up to the note i gave you on monday        preliminary
flow data provided by daren  
please override pop  s daily volume  presently zero  to reflect daily
activity you can obtain from gas control 
this change is needed asap for economics purposes 


In [15]:
st=PorterStemmer()
def clean_text(text):
    text=text.lower()
    text=re.sub(r'\d+','',text)
    text=''.join([i for i in text if i not in string.punctuation])
    tetx=text.strip()
    text=text.replace('subject','').strip()
    text=' '.join([st.stem(word) for word in text.split()])
    return text

In [18]:
df['clean_text']=df['text'].apply(clean_text)
df

,label,text,label_num,clean_text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,enron methanol meter thi is a follow up to the...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,hpl nom for januari see attach file hplnol xl ...
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,neon retreat ho ho ho we re around to that mos...
3,spam,"Subject: photoshop , windows , office . cheap ...",1,photoshop window offic cheap main trend abas d...
4,ham,Subject: re : indian springs\r\nthis deal is t...,0,re indian spring thi deal is to book the teco ...
...,...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0,put the on the ft the transport volum decreas ...
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0,and follow nom hpl can t take the extra mmcf d...
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0,calpin daili ga nomin juli as i mention earlie...
5169,ham,Subject: industrial worksheets for august 2000...,0,industri worksheet for august activ attach are...


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(stop_words='english')

In [25]:
#example
tfidf=vectorizer.fit_transform(['i like apple','i like orange','i like apple and orange'])

pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names_out())

,apple,like,orange
0,0.789807,0.613356,0.000000
1,0.000000,0.613356,0.789807
2,0.619805,0.481334,0.619805


In [30]:
X=vectorizer.fit_transform(df['clean_text'])

In [31]:
y=df['label_num']

In [32]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [33]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(X_train,y_train)
y_pred=nb.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       734
           1       0.99      0.76      0.86       301

    accuracy                           0.93      1035
   macro avg       0.95      0.88      0.91      1035
weighted avg       0.93      0.93      0.93      1035



In [34]:
def predict_email(text):
    global pred
    cleaned_text=clean_text(text)
    vectorized=vectorizer.transform([cleaned_text])
    pred=nb.predict_proba(vectorized)[0]
    if pred[1]>0.3:
        print('This seems like a Spam email')
    else:
        print('This seems like a Genuine email')

In [36]:
exmaple_text='Hello Chatgpt'
predict_email(exmaple_text)

This seems like a Spam email
